In [23]:
## Import relevant modules

from numpy import*
from numpy import loadtxt
from matplotlib.pyplot import*
from iris import*
from iris.analysis import*
import iris.quickplot as quickplot
from iris.coord_categorisation import*
from scipy.stats import*
from scipy.stats.mstats import*
import netCDF4 as nc
import numpy.ma as ma
from subprocess import call
import os

In [24]:
os.chdir("/home/cj362/ENSO_data/notebooks/ENSO_extraction")

In [25]:
# Define Nino3.4 region

north=5
east=-120
south=-5
west=-170

In [26]:
## MIROC6_historical data load

miroc6_historical = iris.load_cube('/home/cj362/ENSO_data/tos/miroc6_historical/tos_Omon_MIROC6_historical_r1i1p1f1_gn_195001-201412_regridded.nc')


In [27]:
## Extract historical data for Nino3.4 region

## MIRCOC6 ##

# Subset files to Nino3.4 region
miroc6_historical_lon = miroc6_historical.intersection(longitude =(west,east))
miroc6_historical_nino34=miroc6_historical_lon.intersection(latitude=(south,north))

## Monthly mean SST across Nino3.4 region 2015-2100 (1032 months = 1032 output values)
miroc6_historical_nino34.coord('longitude').guess_bounds() # Define longitude bounds
miroc6_historical_nino34.coord('latitude').guess_bounds() # Define latitude bounds
miroc6_historical_nino34_mon_mean_grid_area = iris.analysis.cartography.area_weights(miroc6_historical_nino34) # Create grid area
miroc6_historical_nino34_mon_mean = miroc6_historical_nino34.collapsed(['longitude', 'latitude'],\
                              iris.analysis.MEAN, weights=miroc6_historical_nino34_mon_mean_grid_area) # Collape values

# Extracting SST data from the extracted nino3.4 region 
miroc6_historical_nino34_mon_mean_data = miroc6_historical_nino34_mon_mean.data # Extract collapsed file data


In [28]:
## Extract historical data for years (2000-2014) that allow ONI calculation

# Specify years you want (adjusted from code you supplied to appropriate years)
start_year = 2000
end_year = 2014

# print(miroc6_historical_nino34_mon_mean)    # Check the dimensions of employed dataset

# Find which items in this cube that relate to these years
iris.coord_categorisation.add_year(miroc6_historical_nino34_mon_mean, 'time', name='year')      # Comment this line if error comes up saying this oocrdinate already exists
loc1 = np.where((miroc6_historical_nino34_mon_mean.coord('year').points >= start_year) & (miroc6_historical_nino34_mon_mean.coord('year').points <= end_year))

# Extract those years into a new variable called cube1
miroc6_historical_nino34_mon_mean_2000_2014 = miroc6_historical_nino34_mon_mean[loc1]

# print(loc1)       # Prints the month entries for the new cube
# print(cube1.coord('year').points)      # Prints the underlying years that correspond to the months you have printed data for

miroc6_historical_nino34_mon_mean_2000_2014_data = miroc6_historical_nino34_mon_mean_2000_2014.data
# print(miroc6_historical_nino34_mon_mean_2000_to_2014.data) # Prints the SST data entries for the new cube, corresponding to the months that were indexed in 'loc1'

## Save monthly mean SST array as .txt file //     source: https://www.kite.com/python/examples/5522/numpy-save-an-array-to-a-text-file
# Saving format: np.savetxt("outputfilename",np.specified array you want to save)
np.savetxt("/home/cj362/ENSO_data/notebooks/ENSO_extraction/miroc6_data/miroc6_historical_nino34_mon_mean_2000_2014_data.txt",np.array(miroc6_historical_nino34_mon_mean_2000_2014_data))


In [29]:
# Load in climate netCDF files

miroc6_ssp2 = iris.load_cube('/home/cj362/ENSO_data/tos/miroc6_ssp2_45/tos_Omon_MIROC6_ssp245_r1i1p1f1_gn_201501-210012_reggridded.nc')

# print(miroc6_ssp2)

# Define Nino3.4 region
north=5
east=-120
south=-5
west=-170

# Subset files to Nino3.4 region
miroc6_ssp2_lon=miroc6_ssp2.intersection(longitude =(west,east))
miroc6_ssp2_nino34=miroc6_ssp2_lon.intersection(latitude=(south,north))


##  Mean SST for each grid cell between 2015-2100 (714 grid squares = 714 output values)
# miroc6_ssp2_nino34_gridsquare_mean = miroc6_ssp2_nino34.collapsed('time', iris.analysis.MEAN) 
# print(miroc6_ssp2_nino34_gridsquare_mean)
# miroc6_ssp2_nino34_mgridsquare_mean_data = miroc6_ssp2_nino34_gridsquare_mean.data
# print(miroc6_ssp2_nino34_gridsquare_mean_data)

## Monthly mean SST across Nino3.4 region 2015-2100 (1032 months = 1032 output values)
miroc6_ssp2_nino34.coord('longitude').guess_bounds() # Define longitude bounds
miroc6_ssp2_nino34.coord('latitude').guess_bounds() # Define latitude bounds
miroc6_ssp2_nino34_mon_mean_grid_area = iris.analysis.cartography.area_weights(miroc6_ssp2_nino34) # Create grid area
miroc6_ssp2_nino34_mon_mean_2015_2100 = miroc6_ssp2_nino34.collapsed(['longitude', 'latitude'],\
                              iris.analysis.MEAN, weights=miroc6_ssp2_nino34_mon_mean_grid_area) # Collape values

# Extract SST data from extracted Nino3.4 region
miroc6_ssp2_nino34_mon_mean_2015_2100_data = miroc6_ssp2_nino34_mon_mean_2015_2100.data # Extract collapsed file data

# Quick, non-truncated view of all entries within monthly mean SST array:
# for i in range (0, len(miroc6_ssp2_nino34_mon_mean_data)):
    
    # print(miroc6_ssp2_nino34_mon_mean_data[i])
    

# View number of dimensions, shape, size of numpy source: https://note.nkmk.me/en/python-numpy-ndarray-ndim-shape-size/
# array_name.ndim - dimensions
# array_name.shape - shape
# array_name.size - size (multiply sides of the shape)


## Save monthly mean SST array as .txt file source: https://www.kite.com/python/examples/5522/numpy-save-an-array-to-a-text-file
# Format: np.savetxt("outputfilename",np.specified array you want to save)
np.savetxt("/home/cj362/ENSO_data/notebooks/ENSO_extraction/miroc6_data/miroc6_ssp2_nino34_mon_mean_2015_2100_data.txt",np.array(miroc6_ssp2_nino34_mon_mean_2015_2100_data))

 

In [30]:
##  Stitch historical and forecast datasets

# miroc6_ssp2
filenames = ['/home/cj362/ENSO_data/notebooks/ENSO_extraction/miroc6_data/miroc6_historical_nino34_mon_mean_2000_2014_data.txt', '/home/cj362/ENSO_data/notebooks/ENSO_extraction/miroc6_data/miroc6_ssp2_nino34_mon_mean_2015_2100_data.txt']
with open('/home/cj362/ENSO_data/notebooks/ENSO_extraction/miroc6_data/miroc6_stitched_ssp2.txt', 'w') as miroc6_stitched_ssp2:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                miroc6_stitched_ssp2.write(line)
                
# miroc6_ssp5


In [31]:
## ONI Index - Generate 30-year climatology values

## Define function "generate_climatology". Input argument is specified stitched dataset between historical and forecasted data
def generate_climatology (stitched_data):
    
    '''
    This function will use the stitched SST data for the Nino3.4 region from any given model simulation
    to return a dataset comprising values that represent calculated mean SSTs within a 30-year period. 
    These 30-year means are what the three-month-means for each individual month are compared against to
    determine if they are an ENSO event.
    '''
    
    # Initialise climatology list
    climatology_30y_list = []                                            # Make an empty list to begin with. This will allow elements representing 30-year climatology means to be accumulated
                                                                         # List is also easier to append elements to compared to an array. Convert later if necessary
    
    
    # Create for loop for calculating the 30-year climatology values
    for i in range(0, len(stitched_data), 60):                                 # For the entire range of the stitched monthly SST dataset. 
                                                                         # The 60 signifies to the loop to only calculate a new mean every 5 years (60 months/entries)
    
        climatology_30y_entries = stitched_data[i: i+360]                      # Subset the list for the 360 months within the 30-year period you are interested in
                                                                         # A new 30-year climatology will be created every 5 years to accomodate for changes in background warming. 
                                                                         # If not changed every 5 years then the values within those 5 years will be compared against an outdated climate
    
        climatology_mean = mean(climatology_30y_entries)                 # Calculate the mean monthly SST across the filtered 360 months
        climatology_30y_list.append(climatology_mean)                    # Append this value for mean to the list of 30-year climatologies
    
    # Convert list into array
    climatology_30y_array = array(climatology_30y_list)
    
    return climatology_30y_array


## Employ "generate_climatology" function to generate 30-year-climatology mean values for specified model simulations

## miroc6_ssp2
miroc6_stitched_ssp2 = loadtxt('/home/cj362/ENSO_data/notebooks/ENSO_extraction/miroc6_data/miroc6_stitched_ssp2.txt')      # Load in dataet
miroc6_stitched_ssp2.reshape(len(miroc6_stitched_ssp2), 1)                                                                  # Rearrange array into 1D
stitched_data = miroc6_stitched_ssp2                                                                                        # Rename data for use in function
miroc6_stitched_ssp2_climatology_30y_array = generate_climatology(stitched_data)                                            # Call on function to generate 30-year-climatology

## miroc6_ssp5


In [32]:
## ONI Index - Generate three-month-means & compare to 30-year-climatology means. Identify "Episodes"

# # Define "identify_episode" function

def identify_episode (stitched_data, climatology_30y_array):
  
    # Initialise EN and LN episode indication variables
    eln_light = 0 
    lan_light = 0

    # Define print style
    style = "{0:<15}{1:<30}{2:<20}{3:<23}{4:<20}"

    # Print header
    print(style.format("Month Entry", "30-year Climatology Value", "Three-month-mean", "Anomaly (+/-/none)", "Episode indicator"))
    ## For loop for:
    # - Going through all monthly mean SST values
    # - Calculating three-month means
    # - Compare against 30-year climatology mean value
    # - Identifying whether an 'episode' occurs based on all above

    for i in range (180, len(stitched_data)):                                                            # Entry 180 corresponds to January 2015. This range will go until the last month of 2100.
        
        
        ## First 5 years (January 2015 to December 2019)
        if (i<240):                                                                                      # 240 because this entry corresponds to January 2020 when the new 30-year climatology comes into action
                
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3
            climate_thirty = climatology_30y_array[0]
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):                                       # Identifying above-average (El Nino) anomalies (+0.5 degrees Celsius from climatology_30y_array[0])
                
                eln_light = eln_light + 1                                                                # If anomaly detected, light indicating El Nino goes up 1
                if (eln_light >=5):                                                                      # If 5 consecutive anomalies occur, El Nino episode is detected
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))     # Print stand out message
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):                                     # Identifying below-average anomalies (-0.5 degrees Celsius from climatology_30y_array[0])
                
                lan_light = lan_light + 1                                                                # If below-average anomaly detected, light indicating La Nina goes up 1
                if (lan_light >=5):                                                                      # If 5 consecutive anomalies occur, period is indicative of El Nino event
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))     # Print stand out message
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                                                        # If conditions are not met for an anomaly:
                
                eln_light = 0                                                                            # Reset the El Nino light. This permits loop to then find next 5 consecutive months of anomalies if there
                lan_light = 0                                                                            # Reset the La Nina light. This permits loop to then find next 5 consecutive months of anomalies if there
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
            
        ##  Next 5 years (January 2020 to December 2024). Introduces new 30-year climatology
        if (239<i<300):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[1]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
               
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
            
        ##  Next 5 years (January 2025 to December 2029). Introduces new 30-year climatology
        if (299<i<360):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[2]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
            
        ##  Next 5 years (January 2030 to December 2034). Introduces new 30-year climatology
        if (359<i<420):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[3]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))

            
        ##  Next 5 years (January 2035 to December 2039). Introduces new 30-year climatology
        if (419<i<480):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[4]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                
        ##  Next 5 years (January 2040 to December 2044). Introduces new 30-year climatology
        if (479<i<540):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[5]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                
        ##  Next 5 years (January 2045 to December 2049). Introduces new 30-year climatology
        if (539<i<=600):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[6]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                
        ##  Next 5 years (January 2050 to December 2054). Introduces new 30-year climatology
        if (599<i<660):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[7]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
               
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                
        ##  Next 5 years (January 2055 to December 2060). Introduces new 30-year climatology
        if (659<i<720):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[8]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                
        ##  Next 5 years (January 2065 to December 2069). Introduces new 30-year climatology
        if (719<i<780):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3           # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[9]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
               
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
        
        ##  Next 5 years (January 2070 to December 2074). Introduces new 30-year climatology
        if (779<i<840):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3            # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[10]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                
        ##  Next 5 years (January 2075 to December 2080). Introduces new 30-year climatology
        if (839<i<900):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3            # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[11]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                
        ##  Next 5 years (January 2080 to December 2084). Introduces new 30-year climatology
        if (899<i<960):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3            # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[12]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
        
        ##  Next 5 years (January 2085 to December 2089). Introduces new 30-year climatology
        if (959<i<1020):      
            
            three_month_mean = (stitched_data[i-1] + stitched_data[i] + stitched_data[i+1])/3            # Outline process for calculating three-month-mean
            climate_thirty = climatology_30y_array[13]                                                   # Define new 30-year climatology mean
            
            if (three_month_mean >= climatology_30y_array[0]+0.5):     
                
                eln_light = eln_light + 1                              
                if (eln_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "+", "El Nino Episode"))                  
                elif (eln_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "+", "   "))
                    
                    
            elif (three_month_mean <= climatology_30y_array[0]-0.5):   
                
                lan_light = lan_light + 1                              
                if (lan_light >=5):                                    
                    print(style.format(i, climate_thirty, three_month_mean, "-", "La Nina Episode"))                  
                elif (lan_light <5):
                    print(style.format(i, climate_thirty, three_month_mean, "-", "   "))
                    
            else:                                                      
                
                eln_light = 0                                          
                lan_light = 0                                          
                print(style.format(i, climate_thirty, three_month_mean, "none", " "))
                

# MIROC6
stitched_data = miroc6_stitched_ssp2
climatology_30y_array = miroc6_stitched_ssp2_climatology_30y_array
identify_episode(stitched_data, climatology_30y_array)

Month Entry    30-year Climatology Value     Three-month-mean    Anomaly (+/-/none)     Episode indicator   
180            26.4381199635                 23.665791015        -                                          
181            26.4381199635                 24.1868603065       -                                          
182            26.4381199635                 24.6674778857       -                                          
183            26.4381199635                 25.0681705788       -                                          
184            26.4381199635                 25.3742442673       -                      La Nina Episode     
185            26.4381199635                 25.227438115        -                      La Nina Episode     
186            26.4381199635                 24.5085022362       -                      La Nina Episode     
187            26.4381199635                 23.8949322439       -                      La Nina Episode     
188            26.4